<a href="https://colab.research.google.com/github/urban-mobility-rsc/research-v0/blob/main/UMA_CIDADE_indicadores_v1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **0. Preparo do ambiente**

In [ ]:
# essas bibliotecas estão rodando. vê-se no output que há alguns problemas de versão de pacotes que operam muitas vezes dentro destas bibliotecas.
# O erro se evidenciará caso a importação (o passo seguinte), não proceda corretamente. Caso isso ocorra,
# o melhor caminho é desinstalar e instalar novamente algum, o que sempre resolve as eventuais incompatibilidades.
# outras bibliotecas, adicionadas para indicadores pontuais ao longo deste arquivo, devem depois ser verificadas e colocadas aqui, 
# visando uma instalação direta e única de todas as bibliotecas necessárias.

!pip install geobr --quiet
!pip install geopandas --quiet
!pip install pandas --quiet
!pip install shapely --quiet
!pip install osmnx --quiet
!pip install matplotlib==3.5 --quiet
!pip install numpy==1.22 --quiet
!pip install networkx --quiet

In [ ]:
#mesma coisa aqui, importar todas de uma só vez

import geobr
import geopandas as gpd
import pandas as pd
import shapely
import matplotlib
from matplotlib import pyplot as plt
import numpy as np
import networkx as nx
import osmnx as ox

# 0.1 Definições preliminares

In [ ]:
#definição da cidade - ou conjunto de cidades - a ser calculada.
#Usa-se depois com recorrência essa variável, afim de evitar a necessidade de escrever o tempo todo o nome da cidade, estado, país.

cidade = 'Porto Alegre, Rio Grande do Sul, Brazil'

In [ ]:
#Há já em outro arquivo um web scraping que pega a população da cidade a partir de seu nome em outro arquivo de colab.
#para viabilizar os indicadores aqui calculados, foi criada diretamente a variável da população com o dado mais recente do IBGE
pop = 1492530

In [ ]:
# para eventuais necessidades, de forma análoga à população, aqui se tem o código do município no IBGE
code = 4314902

In [ ]:
# download da malha de ruas
graph = ox.graph_from_place(cidade, network_type='all')

In [ ]:
# download dos edifícios da malha. Com esse comando, todos os indicadores que precisam de área construída se tornam possíveis.
tags = {"building": True} 
buildings = ox.geometries_from_place(cidade, tags)

In [ ]:
#área total da cidade [km2]
place = ox.geocode_to_gdf(cidade)
place_proj = ox.project_gdf(place)
area_mun_km = place_proj.area.iloc[0] / 1000000
print(area_mun_km)

493.34050767517596


In [ ]:
#área total da cidade [hectare]
area_mun_ha = area_mun_km * 100
print(area_mun_ha)

49334.0507675176


In [ ]:
#número de domicílios permanentes em Porto Alegre [IBGE]. 
#Com testes de API, optou-se por colocar aqui o valor diretamente a partir do IBGE.
dom = 508456

In [ ]:
# extração dos dados de número de pavimentos de cada edifício da cidade e transformação desse dado em número.
buildings['building:levels'].fillna(3, inplace=True)
building_levels = buildings['building:levels'].dropna().astype(int)

Para todos os indicadores a partir daqui, optou-se pela nomenclatura temporária que corresponde à ordem dos indicadores listados no excel / dicionário. Ao invés de "população por km²", por exemplo, o indicador se chama um_um. Pois é o primeiro indicador do primeiro conjunto de indicadores, no caso, densidade. O segundo indicador do conjunto densidade, "densidade de domicílios", se chama um_dois, e assim por diante.

# **1 - Densidade**

Os indicadores são calculados a partir das constantes previamente definidas

## Geral/Básica

In [ ]:
#1.1 população por km2
um_um = pop / area_mun_km
#um_um = pop / area_mun_ha
print(um_um)

3025.3546521720205


In [ ]:
#1.2 Densidade de domicílios na cidade [household density]
um_dois = dom / area_mun_ha
print(um_dois)

10.306390658980234


In [ ]:
#1.3 ocupação média dos lares [household occupancy]
um_tres = pop / dom
print(um_tres)

2.9354162405399875


In [ ]:
#1.4 Índice de aproveitamento bruto [m²/m²] - Área total construída da cidade dividida pela área total da cidade.

# A area de cada edifício
buildings = buildings.to_crs('EPSG:32722')
buildings['footprint_area'] = buildings['geometry'].area
buildings['building:levels'].fillna(4, inplace=True)

# Soma das áeras dos pavimentos de cada edifício
total_constructed_area = 0
for index, row in buildings.iterrows():
    levels = row['building:levels']
    if levels is not None:
        total_constructed_area += row['footprint_area'] * int(levels)

#transformação da informação "níveis" de cada edifício em número, pois fica cadastrado como texto
buildings['building:levels'] = buildings['building:levels'].astype(float)

# Calculo da área total construída da cidade, multiplicando a projeção de cada edificio pelo seu número de pavimentos
area_total_construida = (buildings['geometry'].area * buildings['building:levels']).sum()

#divisão da área total construída pela área da cidade
um_quatro = area_total_construida / (area_mun_km * 1000000)
print(um_quatro)

0.34271018207562015


In [ ]:
#1.5 Taxa de ocupação bruta [m²/m²] - Soma das projeções de todas as edificações da cidade dividida pela área total da cidade
um_cinco = buildings['geometry'].area.sum() / (area_mun_km * 1000000)
print(um_cinco)

0.11227754848963084


## Áreas públicas, vias e privadas

In [ ]:
#1.6 Soma das áreas abertas e públicas (com rua) [% da área total da cidade]

#dificuldade nesse item, especialmente em calcular área das vias.

tags = {'landuse': 'public'}
gdf_pub = ox.geometries_from_place(cidade, tags)
pub_proj = ox.project_gdf(gdf_pub)
prep_area_pub = pub_proj.area
area_pub = sum(prep_area_pub)
um_seis = area_pub/(area_mun_km*10000)
print(um_seis)

In [ ]:
#1.7 Soma das áreas abertas e públicas (com rua) [área em hectares]
tags = {'leisure': True}

# download dos polígonos de espaços considerados públicos
public_leisure = ox.geometries.geometries_from_place(cidade, tags)

# retirar todas as ruas do GeoDataFrame
public_leisure = public_leisure[public_leisure.geometry.type != 'LineString']

# Calcular a área total
um_sete = public_leisure.area.sum()
print(um_sete)

In [ ]:
# 1.8 Soma das áreas privadas [%]
#não há dados referentes aos lotes de Porto Alegre disponíveis. O osmnx reconhece literalmente 2 lotes na cidade de Porto Alegre toda.
# uma vez que se resolva as áreas privadas ou públicas, o outro estará também resolvido.

tags = {'landuse': 'residential', 'building': 'yes'}
gdf = ox.geometries_from_place(cidade, tags)

# remover polígonos com use residencial que se sobrepõem a vias
gdf = gdf[~gdf.intersects(gdf[gdf["landuse"] == "highway"].unary_union)]
gdf = gdf[~gdf.intersects(gdf[gdf["landuse"] == "waterway"].unary_union)]

um_oito = gdf.area.sum() / (area_mun_km*10000)
print(um_oito)

In [ ]:
# 1.9 Soma das áreas privadas [área em hectare]
#não há dados referentes aos lotes de Porto Alegre disponíveis. O osmnx reconhece literalmente 2 lotes na cidade toda.
# uma vez que se resolva as áreas privadas ou públicas, o outro estará também resolvido.

tags = {'landuse': 'residential', 'building': 'yes'}
buildings = buildings.to_crs('EPSG:32722')
gdf = ox.geometries_from_place(cidade, tags)
um_nove = gdf.area.sum()
print(um_nove)

In [ ]:
#1.10 Áreas abertas e de lazer relativo à área total da cidade [%]
tags = {'leisure': ['bathing_place', 'common', 'garden', 'park', 'track'], 'place': 'square'}
gdf_lazer = ox.geometries_from_place(cidade, tags)
lazer_proj = ox.project_gdf(gdf_lazer)
prep_area_lazer = lazer_proj.area
area_lazer = sum(prep_area_lazer)
um_dez = area_lazer/(area_mun_km*100000)
print(um_dez)

0.1749866333869611


In [ ]:
#1.11 Áreas abertas e de lazer [m²]
tags = {'leisure': ['bathing_place', 'common', 'garden', 'park', 'track'], 'place': 'square'}
gdf_lazer = ox.geometries_from_place(cidade, tags)
lazer_proj = ox.project_gdf(gdf_lazer)
prep_area_lazer = lazer_proj.area
area_lazer = sum(prep_area_lazer)
um_onze = area_lazer
print(um_onze)

8632799.455149328


# **2 - Diversidade / Destinos**

Os primeiros indicadores da lista completa proposta por AUTORES, ANO classificados como diversidade são referentes às:

**A.**   Quantidades de determinados tipos de equipamento (2.1 a 2.8);

**B.** Área relativa de cada equipamento por habitante da cidade - usando aqui projeção das edificações, sem levar em consideração a soma das áreas de diferentes pavimentos (2.9 a 2.16);

**C.** Média da quantidade de população por equipamento (2.17 a 2.24);

**D.**   Distância média dos habitantes até o equipamento mais próximo (2.25 a 2.32) [a fazer];

**E.**   Distância média dos habitantes até os 5 equipamentos mais próximos (2.33 a 2.40) [a fazer].


De maneira complementar, além dos indicadores de *Daily Living Score* (2.41) e caminhabilidade (2.42) consideramos também como indicadores de diversidade alguns dos propostos no projeto de 1000 Cidades Sustentáveis, como percentual da população que reside a até no máximo 500m de determinado tipo de equipamento ou comércio.

## A. QUANTIDADES ABSOLUTAS

In [ ]:
#2.1 Quantidade de escolas na cidade
tags = {'amenity': 'school'}
gdf_escolas = ox.geometries_from_place(cidade, tags)
gdf_escolas.shape
dois_um = len(gdf_escolas)
print(dois_um)

436


In [ ]:
#2.2 Quantidade de pré-escolas e creches na cidade
tags = {'amenity': ['kindergarten']}
gdf_bebes = ox.geometries_from_place(cidade, tags)
gdf_bebes.shape
dois_dois = len(gdf_bebes)
print(dois_dois)

85


In [ ]:
#2.3 Quantidade de universidades na cidade
tags = {'amenity': 'university'}
gdf_uni = ox.geometries_from_place(cidade, tags)
gdf_uni.shape
dois_tres = len(gdf_uni)
print(dois_tres)

31


In [ ]:
#2.4 Quantidade de equipamentos culturais na cidade
tags = {'amenity': ['arts_centre', 'cinema', 'planetarium', 'theatre', 'library']}
gdf_bebes = ox.geometries_from_place(cidade, tags)
gdf_bebes.shape
dois_quatro = len(gdf_bebes)
print(dois_quatro)

114


In [ ]:
#2.5 Quantidade de unidades de saúde na cidade
tags = {'amenity': ['clinic', 'hospital']}
gdf_saude = ox.geometries_from_place(cidade, tags)
gdf_saude.shape
dois_cinco = len(gdf_saude)
print(dois_cinco)

134


In [ ]:
#2.6 Quantidade de supermercados na cidade
tags = {'building': 'supermarket', 'shop': ['convenience', 'supermarket']}
gdf_merc = ox.geometries_from_place(cidade, tags)
gdf_merc.shape
dois_seis = len(gdf_merc)
print(dois_seis)

281


In [ ]:
#2.7 Quantidade de templos e igrejas na cidade
tags = {'amenity': 'place_of_worship'}
gdf_god = ox.geometries_from_place(cidade, tags)
gdf_god.shape
dois_sete = len(gdf_god)
print(dois_sete)

262


In [ ]:
#2.8 Quantidade de shoppings na cidade
tags = {'shop': 'mall'}
gdf_god = ox.geometries_from_place(cidade, tags)
gdf_god.shape
dois_oito = len(gdf_god)
print(dois_oito)

## B. ÁREAS POR HABITANTE

In [ ]:
# 2.9 Área de escolas por habitante [m²/hab]
escolas = ox.geometries_from_place(cidade, {'building': ['school']})
escolas_proj = ox.project_gdf(escolas)
prep_area_escolas = escolas_proj.area
area_escolas = sum(prep_area_escolas)
dois_nove = area_escolas / pop
print(dois_nove)

0.550692816331085


In [ ]:
# 2.10 Área de pre-escolas e creches por habitante [m²/hab]
creches = ox.geometries_from_place(cidade, {'amenity': 'kindergarten'})
creches_proj = ox.project_gdf(creches)
prep_area_creches = creches_proj.area
area_creches = sum(prep_area_creches)
dois_nove = area_creches / pop
print(dois_dez)

0.04201471972484124


In [ ]:
# 2.11 Área de universidades por habitante [m²/hab]
unis = ox.geometries_from_place(cidade, {'amenity': 'university'})
unis_proj = ox.project_gdf(unis)
prep_area_unis = unis_proj.area
area_unis = sum(prep_area_unis)
dois_onze = area_unis / pop
print(dois_onze)

2.053301571434765


In [ ]:
# 2.12 Área de equipamentos culturais por habitante [m²/hab]
cult = ox.geometries_from_place(cidade, {'amenity': ['arts_centre', 'cinema', 'planetarium', 'theatre', 'library']})
cult_proj = ox.project_gdf(cult)
prep_area_cult = cult_proj.area
area_cult = sum(prep_area_cult)
dois_doze = area_cult / pop
print(dois_doze)

0.20189449234888632


In [ ]:
# 2.13 Área de unidades de saúde por habitante [m²/hab]
saude = ox.geometries_from_place(cidade, {'amenity': ['clinic', 'hospital']})
saude_proj = ox.project_gdf(saude)
prep_area_saude = saude_proj.area
area_saude = sum(prep_area_saude)
dois_treze = area_saude / pop
print(dois_treze)

0.4795402983269602


In [ ]:
# 2.14 Área de supermercados por habitante [m²/hab]
mercado = ox.geometries_from_place(cidade, {'building': 'supermarket', 'shop': ['convenience', 'supermarket']})
mercado_proj = ox.project_gdf(mercado)
prep_area_mercado = mercado_proj.area
area_mercado = sum(prep_area_mercado)
dois_catorze = area_mercado / pop
print(dois_catorze)

0.303392643504787


In [ ]:
# 2.15 Área de templos e igrejas por habitante [m²/hab]
templo = ox.geometries_from_place(cidade, {'amenity': 'place_of_worship'})
templo_proj = ox.project_gdf(templo)
prep_area_templo = templo_proj.area
area_templo = sum(prep_area_templo)
dois_quinze = area_templo / pop
print(dois_quinze)

0.10707249115347348


In [ ]:
# 2.16 Área de shoppings por habitante [m²/hab]
shop = ox.geometries_from_place(cidade, {'shop': 'mall'})
shop_proj = ox.project_gdf(shop)
prep_area_shop = shop_proj.area
area_shop = sum(prep_area_shop)
dois_dezesseis = area_shop / pop
print(dois_dezesseis)

## C. QUANTIDADE POR HABITANTE

In [ ]:
#2.17 Quantidade de habitantes por escola
dois_dezessete = pop/dois_um
print(dois_dezessete)

3423.233944954128


In [ ]:
#2.18 Quantidade de habitantes por pré-escolas e creches na cidade
dois_dezoito = pop / dois_dois
print(dois_dezoito)

17559.176470588234


In [ ]:
#2.19 Quantidade de habitantes por universidades na cidade
dois_dezenove = pop / dois_tres
print(dois_dezenove)

In [ ]:
#2.20 Quantidade de habitantes por equipamentos culturais na cidade
dois_vinte = pop / dois_quatro
print(dois_vinte)

In [ ]:
#2.21 Quantidade de habitantes por unidades de saúde na cidade
dois_vinte_um = pop / dois_cinco
print(dois_vinte_um)

In [ ]:
#2.22 Quantidade de habitantes por supermercados na cidade
dois_vinte_dois = pop / dois_seis
print(dois_vinte_dois)

In [ ]:
#2.23 Quantidade de habitantes por templos e igrejas na cidade
dois_vinte_tres = pop / dois_sete
print(dois_vinte_tres)

In [ ]:
#2.24 Quantidade de habitantes por shoppings na cidade
dois_vinte_quatro = pop / dois_oito
print(dois_vinte_quatro)

# **3 - Design / acessibilidade**

## Malha viária - medidas de rede

In [ ]:
#3.1 a 3.5 são adquiridos a partir de estatísticas do grafo

### Estatísticas BÁSICAS da malha/grafo

In [ ]:
#contagem da quantidade de ruas
tags = {'highway': ['motorway', 'trunk', 'primary', 'secondary', 'tertiary']}
gdf_ruas = ox.geometries_from_place(cidade, tags)
gdf_ruas.shape
qtd_ruas = len(gdf_ruas)
print(qtd_ruas)

7183


In [ ]:
# extensão da malha (m)
ext = ox.stats.edge_length_total(graph)
print(ext)

7790020.116000083


In [ ]:
# calcula a área do envelope da malha para gerar medidas de DENSIDADE
gdf = ox.geocode_to_gdf(cidade)
area = ox.project_gdf(gdf).unary_union.area
graph = ox.graph_from_place(cidade, network_type="drive")

In [ ]:
# estatísticas BÁSICAS da malha COM MEDIDAS DE DENSIDADE (precisam da área). Estruturar com Guilherme o uso dos dados gerados aqui em tabelas a serem exportadas.
basic_stats = ox.stats.basic_stats(graph, area=area)
print(basic_stats)

{'n': 20542, 'm': 48804, 'k_avg': 4.751630805179632, 'edge_length_total': 5411966.665999915, 'edge_length_avg': 110.89186677321358, 'streets_per_node_avg': 2.8630123649109143, 'streets_per_node_counts': {0: 0, 1: 3569, 2: 46, 3: 12641, 4: 4208, 5: 73, 6: 4, 7: 1}, 'streets_per_node_proportions': {0: 0.0, 1: 0.17374160257034368, 2: 0.0022393145750170384, 3: 0.6153733813650083, 4: 0.2048486028624282, 5: 0.0035536948690487783, 6: 0.0001947230065232207, 7: 4.868075163080518e-05}, 'intersection_count': 16973, 'street_length_total': 3209541.9620000185, 'street_segment_count': 29398, 'street_length_avg': 109.17552085175925, 'circuity_avg': 1.0391787981619747, 'self_loop_proportion': 0.0008163820668072658, 'node_density_km': 41.638583656554694, 'intersection_density_km': 34.404229403305564, 'edge_density_km': 10970.043168567952, 'street_density_km': 6505.733691167387}


In [ ]:
# estatísticas BÁSICAS da malha COM MEDIDAS DE DENSIDADE em formato lista
pandas.Series(basic_stats)

n                                                                           20542
m                                                                           48804
k_avg                                                                    4.751631
edge_length_total                                                     5411966.666
edge_length_avg                                                        110.891867
streets_per_node_avg                                                     2.863012
streets_per_node_counts         {0: 0, 1: 3569, 2: 46, 3: 12641, 4: 4208, 5: 7...
streets_per_node_proportions    {0: 0.0, 1: 0.17374160257034368, 2: 0.00223931...
intersection_count                                                          16973
street_length_total                                                   3209541.962
street_segment_count                                                        29398
street_length_avg                                                      109.175521
circuity_avg    

In [ ]:
# Explodir os dicionários
#unpack dicts into individiual keys:values

stats = ox.basic_stats(graph, area=area)
for k, count in stats["streets_per_node_counts"].items():
    stats["{}way_int_count".format(k)] = count
for k, proportion in stats["streets_per_node_proportions"].items():
    stats["{}way_int_prop".format(k)] = proportion

# delete the no longer needed dict elements
del stats["streets_per_node_counts"]
del stats["streets_per_node_proportions"]

# load as a pandas dataframe
pandas.DataFrame(pandas.Series(stats, name="value")).round(3)

,value
n,20542.000
m,48804.000
k_avg,4.752
edge_length_total,5411966.666
edge_length_avg,110.892
streets_per_node_avg,2.863
intersection_count,16973.000
street_length_total,3209541.962
street_segment_count,29398.000
street_length_avg,109.176


### Estatísticas AVANÇADAS da malha/grafo

In [ ]:
# Medidas TOPOLÓGICAS específicas da malha. Juntar os dados gerados aqui nas tabelas a serem exportadas.

# 1. Betweenness centrality (centralidade perpasse, entremeamento, "choice")
bc = nx.betweenness_centrality(ox.get_digraph(graph), weight="length")
print(bc)

# 2. Closeness centrality (centralidade proximidade, integração)
nc = nx.closeness_centrality(ox.get_digraph(graph), weight="length")
print(nc)

In [ ]:
# Nós e arestas [sistema viário como grafo]
nodes, edges = ox.graph_to_gdfs(graph)

## Medidas de rede específicas da rede/malha cicloviária


In [ ]:
#grafo da malha cicloviária da cidade
bike = ox.graph_from_place(cidade, network_type='bike')

In [ ]:
#estatisticas das ciclovias
bike_stats = ox.stats.basic_stats(bike)
print(bike_stats)

{'n': 37231, 'm': 86612, 'k_avg': 4.652681904864226, 'edge_length_total': 7326340.913999985, 'edge_length_avg': 84.58805839837419, 'streets_per_node_avg': 2.702291101501437, 'streets_per_node_counts': {0: 0, 1: 8594, 2: 92, 3: 22572, 4: 5767, 5: 191, 6: 13, 7: 2}, 'streets_per_node_proportions': {0: 0.0, 1: 0.23082914775321642, 2: 0.0024710590636834897, 3: 0.6062689694072144, 4: 0.15489780022024657, 5: 0.005130133490908114, 6: 0.0003491713894335366, 7: 5.3718675297467166e-05}, 'intersection_count': 28637, 'street_length_total': 4169046.039000013, 'street_segment_count': 50307, 'street_length_avg': 82.87208617091086, 'circuity_avg': 1.0529855614735821, 'self_loop_proportion': 0.0015703580018685272}


## Tráfego

In [ ]:
# 3.6 proporção de vias arteriais [% do total de vias]
tags = {'highway': 'primary'}
gdf_art = ox.geometries_from_place(cidade, tags)
gdf_art.shape
qtd_art = len(gdf_art)
tres_seis = qtd_art * 100 / qtd_ruas
print(tres_seis)

14.313670673748952


In [ ]:
# 3.7 proporção de vias locais [% do total de vias]
tags = {'highway': 'tertiary'}
gdf_loc = ox.geometries_from_place(cidade, tags)
gdf_loc.shape
qtd_loc = len(gdf_loc)
tres_sete = qtd_loc * 100 / qtd_ruas
print(tres_sete)

41.73888733575622


In [ ]:
# 3.8 proporção de vias p pedestres [% do total de vias]
tags = {'highway': 'pedestrian'}
gdf_ped = ox.geometries_from_place(cidade, tags)
gdf_ped.shape
qtd_ped = len(gdf_ped)
tres_oito = qtd_ped * 100 / qtd_ruas
print(tres_oito)

1.8730779983226167


In [ ]:
#3.9 Quantidade de semáforos
tags = {'highway': 'traffic_signals'}
gdf_semaforo = ox.geometries_from_place(cidade, tags)
gdf_semaforo.shape
tres_nove = len(gdf_semaforo)
print(tres_nove)

1903


In [ ]:
#3.10 Quantidade de faixas de pedestre
tags = {'highway': 'crossing', 'footway':'crossing'}
gdf_semaforo = ox.geometries_from_place(cidade, tags)
gdf_semaforo.shape
tres_dez = len(gdf_semaforo)
print(tres_dez)

2726


## Topografia/terreno

In [ ]:
#3.11 a 3.17 são medidas de topografia. As API testadas até aqui não tiveram resultados adequados. 
#é possível, assim como para população até o momento, pesquisar na web e digitar o valor.

In [ ]:
#3.18 e 3.19 são medidas de rede, ver estatísticas do grafo.

# **4 - SES**

Dados demográficos - idade, raça, sexo - e socioeconômicos: renda, ocupação e escolaridade

# **5 - Infraestrutura**

Este conjunto de indicadores se baseia em sua maioria em dados obtidos através dos dados do censo do IBGE, tabela ENTORNO, como quantidade de domicílios com acesso à agua, a tratamento de esgoto, coleta de lixo, etc.

Há também indicadores agregados da cidade, como emissão de CO2.

Dentre os desafios ao nosso alcance, está a integração dos scripts com os dados disponíveis no GTFS, para análises qualitativas mais complexas de acesso ao transporte público.

In [ ]:
#5.8 Quantidade de pontos de ônibus na cidade
tags = {'highway': ['bus_stop']}
gdf_bus = ox.geometries_from_place(cidade, tags)
gdf_bus.shape
cinco_oito = len(gdf_bus)
print(cinco_oito)

1448


In [ ]:
#5.14 Quantidade de ruas com calçada na cidade
tags = {'highway': ['pedestrian', 'footway', 'path'], 'footway': 'sidewalk', 'sidewalk': ['both', 'left', 'right']}
gdf_bus = ox.geometries_from_place(cidade, tags)
gdf_bus.shape
cinco_catorze = len(gdf_bus)
print(cinco_catorze)

7538
